In [ ]:
import pandas as pd

# Load the CSV file
meta = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Master_dissertation_data/metadata_after_merge.csv",
                    header=0, sep=",", index_col=0)

# List columns before removal
print("Columns before removal:", meta.columns)

# Drop columns that start with "DF." (except "DFPredictions") and those that start with "pANN"
columns_to_remove = [col for col in meta.columns if (col.startswith("DF.") and col != "DFPredictions") or col.startswith("pANN")]
meta = meta.drop(columns=columns_to_remove)

# List columns after removal
print("Columns after removal:", meta.columns)

# Save the cleaned metadata
meta.to_csv(r"/content/drive/MyDrive/Colab Notebooks/Master_dissertation_data/cleaned_metadata.csv", index=True)

print("Cleaned metadata saved successfully!")


<ipython-input-2-ae67f338fdbf>:4: DtypeWarning: Columns (10,11,12,13,14,15,16,17,18,19,20,21,30,31,32,33,38,39,40,41,43,44,45,46,48,49,50,51,59,60,61,62,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,84,86,87,88,89,91,93,94,95,96,98,99,100,101,103,104,105,106,108,109,110,111,113,114,115,116,118,119,120,121,123,124,125,126,128,129,130,131,133,134,135,136,138,139,140,141,142,143,144,146,147,148,149,151,152,153,154,156,157,158,159,161,162,163,164,166,167,168,169,171,172,173,174,176,177,178,179,181,182,183,184,186,187,188,189,191,192,193,195,196,197,198,200,201,202,203,205,206,207,208,210,211,212,213,215,216,217,218,220,221,222,223,225,226,227,228,230,231,232,233,235,236,237,238,240,241,242,243,245,246,247,248) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Master_dissertation_data/metadata_after_merge.csv",


Columns before removal: Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO',
       'nFeature_HTO', 'nCount_ADT', 'nFeature_ADT', 'source_matrix',
       'HTOmaxval', 'HTODemux.maxID',
       ...
       'pANN_0.25_0.22_200', 'DF.classifications_0.25_0.22_200',
       'DF.classifications_0.25_0.22_182', 'DF.classifications_0.25_0.22_3005',
       'DF.classifications_0.25_0.22_2740', 'pANN_0.25_0.21_160',
       'DF.classifications_0.25_0.21_160', 'DF.classifications_0.25_0.21_139',
       'DF.classifications_0.25_0.21_2396',
       'DF.classifications_0.25_0.21_2079'],
      dtype='object', length=248)
Columns after removal: Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO',
       'nFeature_HTO', 'nCount_ADT', 'nFeature_ADT', 'source_matrix',
       'HTOmaxval', 'HTODemux.maxID', 'HTODemux.secondID', 'HTODemux.margin',
       'HTODemux.classification', 'HTODemux.classification.global',
       'HTODemux.ID', 'MultiSeq.ID', 'MultiSeq.classification',
       'Fre

In [2]:
import pandas as pd

# Load the CSV
patientoverview = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Master_dissertation_data/Metadata_table_clean.csv",
                              header=0, sep=";")

In [4]:
patientoverview.head()

# Create the 'subsample' column by extracting the number and formatting it correctly
patientoverview['subsample'] = patientoverview['MultiSeq.ID'].str.extract(r'(\d+)')  # Extract the number
patientoverview['subsample'] = patientoverview['subsample'].apply(lambda x: f"#{x}" if pd.notna(x) else x)  # Format as '#X'

# Display the first few rows to verify
print(patientoverview[['MultiSeq.ID', 'subsample']].head())


  MultiSeq.ID subsample
0    Hashtag1        #1
1    Hashtag2        #2
2    Hashtag7        #7
3    Hashtag5        #5
4    Hashtag5        #5


In [6]:
patientoverview = patientoverview.drop(columns=['MultiSeq.ID'])

In [7]:
patientoverview.head()

,day,exp,Patient,Sorting,sample ID,sample type,sample nr,Nr cells sorted,subsample
0,1,SAB001,HC1,CD34+,230309-1218,BM,1,1 993,#1
1,1,SAB001,HC2,CD34+,230313-3523,BM,2,3 693,#2
2,1,SAB001,HC5,CD34+,230309-3012,BM,7,1 705,#7
3,1,SAB001,AML0028,CD34+,AMBM120215,BM,5,60 893,#5
4,1,SAB002,AML0028,CD34+,AMBM120215,BM,5,100 000,#5


In [14]:
import pandas as pd

meta_old = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Master_dissertation_data/cleaned_metadata.csv", header=0, sep=",", index_col=0)

# Save the original index
meta_old_index = meta_old.index

# Work on a copy of meta_old
meta = meta_old.copy()

# Extract SABxxx from the 'orig.ident' column in meta
meta['exp'] = meta['orig.ident'].str.extract(r'(SAB\d{3})')

# Merge without duplicating rows (ensuring no row expansion)
meta = meta.merge(patientoverview[['exp', 'day', 'Sorting']].drop_duplicates(), on='exp', how='left')

# Ensure 'subsample' exists and extract only the first two characters for comparison
meta['subsample_match'] = meta['subsample'].str[:2]  # Extract only the first two characters (e.g., "#7")
patientoverview['subsample_match'] = patientoverview['subsample'].str[:2]  # Do the same for patientoverview

# Convert patientoverview to a dictionary for faster lookups
patient_dict = {(row['exp'], row['subsample_match']): row for _, row in patientoverview.iterrows()}
exp_dict = {row['exp']: row for _, row in patientoverview.iterrows()}  # Lookup based on exp only

# Ensure new columns exist before filling them in the loop
for col in ["Patient", "sample ID", "sample type", "sample nr", "Nr cells sorted"]:
    meta[col] = pd.NA  # Initialize columns to avoid missing key errors

# Loop through each row in meta and assign matching values
for index, row in meta.iterrows():
    exp_val = row['exp']
    subsample_val = row['subsample_match']  # Use the extracted two-character match

    # If subsample is "Negative" or "Doublet", keep NA values
    if subsample_val in ["Negative", "Doublet"]:
        continue  # Skip to the next iteration

    key = (exp_val, subsample_val)

    # First try exact match on exp + subsample_match
    if key in patient_dict:
        patient_row = patient_dict[key]
    # If subsample is NaN, use only exp to fetch values
    elif pd.isna(subsample_val) and exp_val in exp_dict:
        patient_row = exp_dict[exp_val]
    else:
        continue  # Skip if no matching data is found

    # Copy values to the meta table
    meta.at[index, "Patient"] = patient_row["Patient"]
    meta.at[index, "sample ID"] = patient_row["sample ID"]
    meta.at[index, "sample type"] = patient_row["sample type"]
    meta.at[index, "sample nr"] = patient_row["sample nr"]
    meta.at[index, "Nr cells sorted"] = patient_row["Nr cells sorted"]

# Remove temporary 'subsample_match' columns
meta = meta.drop(columns=['subsample_match'])
patientoverview = patientoverview.drop(columns=['subsample_match'])

# Check row count before reassigning index
if len(meta) == len(meta_old_index):
    meta.index = meta_old_index  # Restore the original index safely
else:
    print(f"Warning: Row count mismatch! meta has {len(meta)} rows, but the original had {len(meta_old_index)}.")

# Print unique exp values with the newly added columns for verification
print(meta[['exp', 'subsample', 'Patient', 'sample ID', 'sample type', 'sample nr', 'Nr cells sorted', "Sorting", "day"]].drop_duplicates())

# Save updated metadata
meta.to_csv(r"/content/drive/MyDrive/Colab Notebooks/Master_dissertation_data/updated_metadata.csv", index=True)

print("Updated metadata saved successfully!")


<ipython-input-14-409c6eeeb2f6>:3: DtypeWarning: Columns (10,11,12,13,14,15,16,17,18,19,20,21,39) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_old = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Master_dissertation_data/cleaned_metadata.csv", header=0, sep=",", index_col=0)


                              exp subsample  Patient    sample ID sample type  \
SAB001_AAACGGGGTGTGGCTC-1  SAB001    #7_3,3     HC5   230309-3012          BM   
SAB001_AAACGGGGTAGGCATG-1  SAB001    #1_2,2      HC1  230309-1218          BM   
SAB001_AAACCTGAGTCTCCTC-1  SAB001    #2_0,0      HC2  230313-3523          BM   
SAB001_AAACCTGAGAAACCTA-1  SAB001    #5_1,1  AML0028   AMBM120215          BM   
SAB002_AAACCTGAGAACAACT-1  SAB002       NaN  AML0028   AMBM120215          BM   
SAB003_AAACCTGAGAGCAATT-1  SAB003       NaN  AML0028   AMBM120215          BM   
SAB004_AAACCTGAGACAAGCC-1  SAB004       NaN  AML1744   AMBM101017          BM   
SAB005_AAACCTGAGACTAAGT-1  SAB005       NaN  AML1744   AMBM101017          BM   
SAB006_AAACGGGCACAGACTT-1  SAB006    #3_0,0  AML0009  AMPB170815           PB   
SAB006_AAACCTGAGTAAGTAC-1  SAB006    #5_1,1  AML0028   AMBM120215          BM   
SAB006_AAACCTGAGAACAATC-1  SAB006    #4_2,2  AML0020   AMAF090715          AF   
SAB007_AAACCTGAGATATGCA-1  S

In [13]:
meta_old.head()

,exp,Patient,Sorting,sample ID,sample type,sample nr,Nr cells sorted,MultiSeq.ID
day,,,,,,,,
1,SAB001,HC1,CD34+,230309-1218,BM,1,1 993,Hashtag1
1,SAB001,HC2,CD34+,230313-3523,BM,2,3 693,Hashtag2
1,SAB001,HC5,CD34+,230309-3012,BM,7,1 705,Hashtag7
1,SAB001,AML0028,CD34+,AMBM120215,BM,5,60 893,Hashtag5
1,SAB002,AML0028,CD34+,AMBM120215,BM,5,100 000,Hashtag5
